In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import os

In [2]:
pd.options.display.max_columns = None

### Import Data

In [3]:
path = r'/Users/henning/Documents/Data Analytics Portfolio/Instacart Project - ICP'

In [4]:
df_icp = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'merged_data_set.pkl'))

### Creating price categories for the products

In [5]:
# Set three prices ranges with the log-function
df_icp.loc[df_icp['prices'] > 15, 'price_range'] = 'High-range product'
df_icp.loc[(df_icp['prices'] <= 15) & (df_icp['prices'] > 5), 'price_range'] = 'Mid-range product' 
df_icp.loc[df_icp['prices'] <= 5, 'price_range'] = 'Low-range product'

/var/folders/5_/pv74frmn4fx0_vntxlc18yhm0000gn/T/ipykernel_85385/366770025.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'High-range product' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_icp.loc[df_icp['prices'] > 15, 'price_range'] = 'High-range product'


In [6]:
# checking 
df_icp['price_range'].value_counts(dropna = False)

price_range
Mid-range product     22825196
Low-range product     10557208
High-range product      429816
NaN                       7488
Name: count, dtype: int64

In [7]:
df_icp.sample(3)

,reordered,customer,order_number,order_weekday,order_hour_of_day,days_since_last_order,product_name,prices,department,aisle,gender,state,age,date_joined,civil_status,income,price_range
11246061,0,166288,11,1,11,18.0,Milano Mint Cookies,2.5,snacks,cookies cakes,Male,North Carolina,35,10/30/2017,married,89691,Low-range product
7254288,1,21982,6,4,11,30.0,Organic White Chocolate Peanut Butter Cups,5.0,snacks,candy chocolate,Male,Alabama,70,3/1/2017,divorced/widowed,80806,Low-range product
7863051,1,67043,22,4,11,4.0,Organic Mini Homestyle Waffles,2.3,frozen,frozen breakfast,Male,North Dakota,56,8/5/2018,married,55454,Low-range product


### Creating Column with the number of orders a customer has placed so far

In [8]:
df_icp['max_order'] = df_icp.groupby(['customer'])['order_number'].transform(np.max)

/var/folders/5_/pv74frmn4fx0_vntxlc18yhm0000gn/T/ipykernel_85385/1867003430.py:1: FutureWarning: The provided callable <function amax at 0x111b25620> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_icp['max_order'] = df_icp.groupby(['customer'])['order_number'].transform(np.max)


In [9]:
df_icp.sample(3)

,reordered,customer,order_number,order_weekday,order_hour_of_day,days_since_last_order,product_name,prices,department,aisle,gender,state,age,date_joined,civil_status,income,price_range,max_order
8435497,0,43351,7,3,8,5.0,Red Grapefruit,2.4,produce,fresh fruits,Female,Alabama,63,2/6/2019,married,161095,Low-range product,94
13376691,1,125186,18,6,14,2.0,Steak Sauce,13.6,pantry,marinades meat preparation,Female,Oregon,51,1/9/2020,married,51402,Mid-range product,18
15694293,1,107238,7,2,12,27.0,Club Soda,8.5,beverages,soft drinks,Male,Minnesota,64,6/11/2019,divorced/widowed,77841,Mid-range product,13


### Creating Column for low and high spender. 

If the average price of all orders of a customer is higher than ..., then this customer is considered a high spender.

In [10]:
# adding a column with the total cost of each order
df_icp['order_total'] = df_icp.groupby(['customer', 'order_number'])['prices'].transform(np.sum)

/var/folders/5_/pv74frmn4fx0_vntxlc18yhm0000gn/T/ipykernel_85385/2219699636.py:2: FutureWarning: The provided callable <function sum at 0x111b24cc0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  df_icp['order_total'] = df_icp.groupby(['customer', 'order_number'])['prices'].transform(np.sum)


In [11]:
# creating a column with the average spend per order of a customer
df_icp['median_price_per_order'] = df_icp.groupby(['customer'])['order_total'].transform(np.median)

/var/folders/5_/pv74frmn4fx0_vntxlc18yhm0000gn/T/ipykernel_85385/2063419755.py:2: FutureWarning: The provided callable <function median at 0x113170680> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_icp['median_price_per_order'] = df_icp.groupby(['customer'])['order_total'].transform(np.median)


In [12]:
df_icp['median_price_per_order'].max()

718.6

In [13]:
df_icp['median_price_per_order'].min()

1.0

In [14]:
df_icp['median_price_per_order'].describe()

count    3.381971e+07
mean     1.203322e+02
std      5.892960e+01
min      1.000000e+00
25%      7.800000e+01
50%      1.109000e+02
75%      1.522000e+02
max      7.186000e+02
Name: median_price_per_order, dtype: float64

In [15]:
# creating the column for the 2 groups, desided on the threshold of 78 ~ 25% median
df_icp.loc[df_icp['median_price_per_order'] < 78, 'spend_flag'] = 'Low spender'
df_icp.loc[df_icp['median_price_per_order'] >= 78, 'spend_flag'] = 'High spender'

/var/folders/5_/pv74frmn4fx0_vntxlc18yhm0000gn/T/ipykernel_85385/2072758051.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Low spender' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_icp.loc[df_icp['median_price_per_order'] < 78, 'spend_flag'] = 'Low spender'


In [16]:
df_icp['spend_flag'].value_counts(dropna = False)

spend_flag
High spender    25389756
Low spender      8429952
Name: count, dtype: int64

### Creating a Region column

In [17]:
# Set four Regions with the log-function
df_icp.loc[df_icp['state'].isin(['Maine', 'New Hampshire', 'Vermont', 'massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']), 'region'] = 'Region 1 Northeast'
df_icp.loc[df_icp['state'].isin(['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Iowa', 'Missouri']), 'region'] = 'Region 2 Midwest' 
df_icp.loc[df_icp['state'].isin(['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']), 'region'] = 'Region 3 South'
df_icp.loc[df_icp['state'].isin(['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']), 'region'] = 'Region 4 West'

/var/folders/5_/pv74frmn4fx0_vntxlc18yhm0000gn/T/ipykernel_85385/1119219788.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Region 1 Northeast' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_icp.loc[df_icp['state'].isin(['Maine', 'New Hampshire', 'Vermont', 'massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']), 'region'] = 'Region 1 Northeast'


In [18]:
# checking 
df_icp['region'].value_counts(dropna = False)

region
Region 3 South        11264099
Region 4 West          8652562
Region 2 Midwest       7256695
Region 1 Northeast     5296514
NaN                    1349838
Name: count, dtype: int64

### Export

In [19]:
df_icp.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'final_data.pkl'))